**Kaggle team name**: 🐧06🦠 3-FancyBacteria-Cui,Rivas
1. Format: `🐧[BB Team#]-[Your fancy Kaggle group name]-[Students' names]`. 
  1. Eg. `🐧A-Heros-Fleischer,Melnikov`, where 🐧 identifies JHU and `A` identifies the Canvas group code

Your private LB score must be reproducible with this Colab. Seed all [RNG](https://en.wikipedia.org/wiki/Random_number_generation). Don't exceed runtime quota.

<small><hr style="margin:0;background-color:silver"><font color=gray>Notebook author: <a href="https://www.linkedin.com/in/olegmelnikov/" target="_blank">Oleg Melnikov</a>, ©<a href="https://apps.ep.jhu.edu/course-homepages/3765-605-742-deep-neural-networks" target="_blank">JHU</a> 2021 onwards</font></small>

<hr color=darkblue>

# **5🏆🦠Bacteria**

<details><summary><font color=darkblue>More info and Kaggle API instructions</font></summary>

[Kaggle competition URL](https://www.kaggle.com/c/22822bacteria/rules). See competition rules, submission, grading, dataset, and performance metric. The **starter code** below produces a baseline model, which you should beat, while respecting the competition rules. Your code starts after the timer. This is your baseline model. Seed all [RNG](https://en.wikipedia.org/wiki/Random_number_generation) for reproducibility!

**Instructions for enabling Kaggle API in Colab**:
1. Accept competition rules before running [Kaggle API](https://github.com/Kaggle/kaggle-api#api-credentials). [Loading Kaggle dataset example](https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab)
1. In your Kaggle Account, [Create API Token](https://github.com/Kaggle/kaggle-api#api-credentials) and save the resulting **kaggle.json** file to the [root of your Google Drive](https://drive.google.com/drive/u/0/my-drive) 
2. In Colab, open **Files** panel 🗀 (on the left) and click gray folder icon <font color=gray>🖿</font> to mount your Google drive

Your Kaggle/Google Drive credentials are secure; and Colab's kaggle.json only lasts a Colab session.

</details>


In [ ]:
from google.colab import drive; drive.mount('/content/drive')   # OK to enable, if kaggle.json is stored in Google Drive

Mounted at /content/drive


In [ ]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                               # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log       # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                  # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 22822bacteria # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log              # download competition dataset as a zip file
!unzip -o *.zip >> log                            # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show           # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 22822bacteria
100% 41.6M/41.6M [00:00<00:00, 163MB/s]
Using competition: 22822bacteria
 teamId  teamName                                         submissionDate       score    
-------  -----------------------------------------------  -------------------  -------  
8886716  Steven Nilla                                     2022-07-15 22:23:26  0.99720  
8878002  🐧4-MenOfCulture-Nathanson,Nilla                  2022-07-17 00:10:09  0.99680  
8881482  🐧5-JointChiefsOfStaph-Bollineni, Na              2022-07-17 00:52:30  0.99680  
8235147  JE-TBD-Barrett,Natali                            2022-03-10 20:16:29  0.99660  
8258737  JF-BacteriaBattalion-Brodsky,Khan                2022-03-09 02:19:41  0.99640  
8296182  JD-Germophobe-Hernandez-Merran-Rumman            2022-03-14 00:40:52  0.99640  
8877868  🐧6-TheTriumvirate-Cahill,Rice,Sahi               2022-07-17 20:22:39  0.99600  
8265342  JA-Culture-Mendol

In [ ]:
!pip -q install -U tensorflow_addons > log

In [ ]:
%%time
%%capture
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all" 
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, seaborn as sns, tensorflow_addons as tfa
from sklearn.preprocessing import PolynomialFeatures
import tensorflow as tf, tensorflow.keras as keras
from keras.layers import Flatten, Dense
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv', index_label='id') # rounds values to 2 decimals

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

np.set_printoptions(linewidth=100, precision=2, edgeitems=2, suppress=True)
pd.set_option('max_columns', 20, 'precision', 2, 'display.max_rows', 4)

CPU times: user 4.93 s, sys: 452 ms, total: 5.38 s
Wall time: 6.79 s


In [ ]:
df = pd.read_csv('XY_Bacteria.csv'); df

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,y
0,10,32,41,39,77,122,55,81,58,31,...,38,88,80,20,36,31,32,32,10,NaN
1,10,31,52,165,225,221,150,143,48,31,...,111,125,143,159,69,45,32,71,10,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99998,10,31,66,107,142,155,142,107,66,31,...,93,66,93,93,66,31,31,31,10,4.0
99999,10,32,41,21,45,89,45,60,67,31,...,97,82,74,80,52,31,32,7,10,2.0


In [ ]:
df.info()   # observe datatypes and any missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 287 entries, A0T0G0C10 to y
dtypes: float64(1), int64(286)
memory usage: 219.0 MB


In [ ]:
vX = df.query('y!=y').drop('y', axis=1)    # slice a test sample
tXY = df.query('y==y')                     # slice training sample
tX, tY = tXY.drop('y', axis=1), tXY.y.astype(int)      # split into training I/O
vX  # test inputs
tX  # train inputs
print(tY.tolist()[:50]) # train outputs

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,10,32,41,39,77,122,55,81,58,31,...,27,38,88,80,20,36,31,32,32,10
1,10,31,52,165,225,221,150,143,48,31,...,76,111,125,143,159,69,45,32,71,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9998,10,31,66,107,142,156,143,107,66,31,...,420,296,303,421,302,306,31,31,31,10
9999,10,31,66,107,142,155,142,108,66,31,...,66,429,307,93,93,66,31,31,31,10


,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
10000,10,7,48,74,119,110,127,86,66,7,...,223,319,233,361,439,368,179,174,199,10
10001,10,31,66,107,142,156,142,108,66,31,...,66,539,302,93,424,434,31,313,31,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99998,10,31,66,107,142,155,142,107,66,31,...,66,93,66,93,93,66,31,31,31,10
99999,10,32,41,21,45,89,45,60,67,31,...,52,97,82,74,80,52,31,32,7,10


[3, 3, 4, 0, 4, 3, 4, 4, 3, 3, 2, 2, 4, 2, 2, 4, 4, 2, 2, 0, 2, 3, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 4, 2, 3, 3, 2, 0, 1, 4, 4, 2, 3, 4, 3, 0, 3, 2, 2, 0]


In [ ]:
tmr = Timer()

⏳ started. You have 60 sec. Good luck!


<hr color=red>

<font size=5>⏳</font> <strong><font color=orange size=5>Your Code, Documentation, Ideas and Timer - All Start Here...</font></strong>

**Student's Section** (between ⏳ symbols): add your code and documentation here.

## **Task 1. Preprocessing Pipeline**
 
Explain elements of your preprocessing pipeline i.e. feature engineering, subsampling, clustering, dimensionality reduction, etc. 
1. Why did you choose these elements? (Something in EDA, prior experience,...? Btw, EDA is not required)
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

1. We used histograms to investigate the data distributions.We discovered that most features are not normally distributed, so we first tried out PowerTransformer to normalize the data. We also used other transformer tools to preprocess the data, such as polynomial fitting. PCA logic was also implemented to clean the features and reduce training time. We implemented an outlier detection code to remove the outlier rows from the dataset.
<br></br>

2. We first used "accuracy" and "loss" metrics to evaluate the prediction model. However, as we experimented with different hyperparameters, we discovered that the "accuracy" score severely mismatched the actual prediction result. As a result, we used "MSE" and "Sparse Categorical Accuracy" to measure the performance.
<br></br>

3. Most of the preprocessing techniques we tried failed to improve the model. We've tried PowerTransformer, QuantileTransformer, Normalization, SplineTransformer, Scaling, Outlier removing, and PCA. We weren't able to run the polynomial fit with a degree of 2 as the collab crashed because of ram limitation. In the end, we used original data for training.

## **Task 2. Modeling Approach**
Explain your modeling approach, i.e. ideas you tried and why you thought they would be helpful. 

1. How did these decisions guide you in modeling?
1. How do you evaluate the effectiveness of these elements? 
1. What else have you tried that worked or didn't? 

**Student's answer:**

1. We implemented exponential decay to try out different learning rates for the DNN model. Different regularization, initializer, and optimizer were also experimented with various settings to improve the model performance. We used the partial function to organize the DNN code to maintain the cleanness of the code. Early stopping callback is implemented to reduce the DNN training time.
<br></br>
2. As mentioned in the Task 1 section, we first used "accuracy" and "loss" metrics to evaluate the prediction model. However, we discovered that the "accuracy" score severely mismatched the actual prediction result and the scores on the Kaggle. Therefore, we used "MSE" and "Sparse Categorical Accuracy" to measure the performance in the end.
<br></br>
3. Relu and ELU regularizers didn't seem to perform well in the bacteria dataset. He initializer did not outperform the Lecun normal. Early stopping callback reduces the training time significantly, but sometimes the training stops too early. As a result, it creates a flawed model that has very low prediction accuracy.

#The following code is not used as it does not improve the performance of the prediction model.

In [ ]:
# # standard min max scaling won't work
# from sklearn.preprocessing import PowerTransformer
 
# pt = PowerTransformer()
 
# df_scaled = pt.fit_transform(tXY.loc[:, tXY.columns != 'y'].to_numpy())
# df_scaled = pd.DataFrame(df_scaled, columns=tXY.columns[tXY.columns != 'y'])

# --------------------------
# # Try quantile transformer
# from sklearn.preprocessing import QuantileTransformer
# qt = QuantileTransformer(output_distribution='normal', random_state=0)

# df_scaled = pt.fit_transform(tXY.loc[:, tXY.columns != 'y'].to_numpy())
# df_scaled = pd.DataFrame(df_scaled, columns=tXY.columns[tXY.columns != 'y'])

# --------------------------

# # Try normalize
# from sklearn.preprocessing import Normalizer
# normal = Normalizer(norm='max')

# df_scaled = normal.fit_transform(tX.loc[:, tX.columns != 'y'].to_numpy())
# df_scaled = pd.DataFrame(df_scaled, columns=tX.columns[tX.columns != 'y'])
# tX = df_scaled

# --------------------------
# # Try SplineTransformer
# from sklearn.preprocessing import SplineTransformer
# spline = SplineTransformer(degree=2, n_knots=3)
# df_scaled = spline.fit_transform(tXY.loc[:, tXY.columns != 'y'].to_numpy()) * 100
# #df_scaled = pd.DataFrame(df_scaled, columns=tXY.columns[tXY.columns != 'y'])
# df_scaled = pd.DataFrame(df_scaled)


# --------------------------
# # show histogram
# df_scaled.hist(column='A0T0G0C10')
# df.hist(column='A0T0G0C10')

# df_scaled.hist(column='A10T0G0C0')
# df.hist(column='A10T0G0C0')
# df_scaled = df_scaled * 100

# --------------------------
# Use PCA to drop the features - Faster training time but overfitting
# from sklearn.decomposition import PCA
# # Get reduced dimensionality 
# pca = PCA(n_components=0.99)
# X_reduced = pca.fit_transform(tX)
# X_reduced.shape
# dfX_est = tX.iloc[:,:X_reduced.shape[1]] # 0.95 threshold
# # Assign reduced data
# dfX_est[:] = X_reduced
# --------------------------
# # Try Polynomial fit - Can't use it because the ram limitation.
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(degree=2)
# df_scaled = poly.fit_transform(dfX_est.loc[:, dfX_est.columns != 'y'].to_numpy())# * 100
# df_scaled = pd.DataFrame(df_scaled, columns=tXY.columns[tXY.columns != 'y'])
# df_scaled = pd.DataFrame(df_scaled)
# dfX_est.head()

# -------------------------
# Inspired by Carlos' code

# def outliers(df, ft):
#   # q1 = df[ft].quantile(.05)
#   # q3 = df[ft].quantile(.95)
#   q1 = df[ft].quantile(.01)
#   q3 = df[ft].quantile(.99)

#   iqr = q3 - q1
  
#   low_bound = q1 - 1.5 * iqr
#   upp_bound = q3 + 1.5 * iqr

#   ls = df.index[(df[ft] < low_bound) | (df[ft] > upp_bound)]

#   return ls

# def remove(df, ls):
#   ls = sorted(set(ls))
#   df = df.drop(ls)
#   return df

# index_list = []

# for col in df.columns[:-1]:
#   index_list.extend(outliers(df, col))

In [ ]:
# Learning rate for exponential decay
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponential_decay_fn

# Assign exponential decay function to use in the DNN callbacks
exponential_decay_fn = exponential_decay(lr0=0.001, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

# Regularization with SELU and Lecun Normal
from functools import partial

# Use partial to wrap the callback
RegularizedDense = partial(keras.layers.Dense,
                           activation="selu",
                           kernel_initializer="lecun_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

# Early Stopping - NOT USED for final model
#es = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='auto', verbose=1, restore_best_weights=True)


Below is a baseline model that produces the result on Kaggle leaderboard (LB).

In [ ]:
%%time

tf.random.set_seed(0)   # always seed your experiments
Init = keras.initializers.LecunNormal(seed=0) # Use Lecun Normal

tX_in = tXY.drop('y', axis=1)[:50000]
tY_in = tXY.y[:50000]

m = keras.models.Sequential([
    Flatten(input_shape=[tX_in.shape[1]], name='input'),
    Dense(200, activation="selu", kernel_initializer=Init, name='hidden', kernel_constraint=keras.constraints.max_norm(1.)),
    RegularizedDense(300),
    RegularizedDense(100),
    Dense(len(tY.unique()), activation='softmax', kernel_initializer=Init, name='output',kernel_constraint=keras.constraints.max_norm(1.))])
m.summary()
m.compile(loss="sparse_categorical_crossentropy", metrics=['mse', 'sparse_categorical_accuracy'], optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001))
hist = m.fit(tX_in, tY_in, callbacks=[lr_scheduler], epochs=6, validation_split=0.15, batch_size=128)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Flatten)             (None, 286)               0         
                                                                 
 hidden (Dense)              (None, 200)               57400     
                                                                 
 dense (Dense)               (None, 300)               60300     
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 output (Dense)              (None, 5)                 505       
                                                                 
Total params: 148,305
Trainable params: 148,305
Non-trainable params: 0
_________________________________________________________________
Epoch 1/6
333/333 [==============================]

In [ ]:
pY = pd.DataFrame(np.argmax(m.predict(vX), axis=-1), index=vX.index+1, columns=['y'])
ToCSV(pY.round(0).astype(int), 'MySubmission')
print('Observed  y distribution: ', list((tXY.y.value_counts()/len(tXY)).round(3)))                # distribution of training target level
print('Predicted y distribution: ', list((pY.round(0).astype(int).value_counts()/len(pY)).round(3))) # distribution of test target level

Observed  y distribution:  [0.303, 0.302, 0.296, 0.05, 0.048]
Predicted y distribution:  [0.313, 0.313, 0.293, 0.041, 0.039]


# **References:**

1. Géron, A. (2019). Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems (2nd ed.). O’Reilly Media.

2. Brownlee, J. (2020, August 25). Use Early Stopping to Halt the Training of Neural Networks At the Right Time. Machine Learning Mastery. https://machinelearningmastery.com/how-to-stop-training-deep-neural-networks-at-the-right-time-using-early-stopping/

3. K. (2021a, December 20). Pandas Filter Rows with NAN Value from DataFrame Column. Spark by {Examples}. https://sparkbyexamples.com/pandas/pandas-filter-rows-with-nan-value-from-dataframe-column/#:%7E:text=You%20can%20filter%20out%20rows,represented%20as%20None%20or%20NaN.

4. 6.3. Preprocessing data. (n.d.). Scikit-Learn. https://scikit-learn.org/stable/modules/preprocessing.html#generating-polynomial-features



<font size=5>⌛</font> <strong><font color=orange size=5>Do not exceed competition's runtime limit!</font></strong>

<hr color=red>


In [ ]:
tmr.ShowTime()    # measure Colab's runtime. Do not remove. Keep as the last cell in your notebook.

Runtime is 38 sec


## 💡**Starter Ideas**

**Model**
1. Tune model hyperparameters, batch size, optimizer, NN layers

**Features**
1. Try to linear and non-linear feature normalization: shift/scale, log, divide features by features (investigate scatterplot matrix)
1. Try higher order feature interactions and polynomial features on a small subsample. Then identify key features or select key principal components. The final model can be trained on a larger or even full training sample. You can use [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the feature set

**Training observations**
1. Try clustering similar observations into fewer representative observations. 
1. You may also try dimension reduction methods (eg. PCA) on the transposed data matrix (if it has scaled numeric features).
1. Look for and deal with outliers or influential points in the training set
1. Deal with **imbalanced sample**: oversample smaller class, or undersample larger class, or provide observation weights or provide class weights, or seek a suitable loss function
1. Investigate distributions of features. Any missing values? Any zero values?

**Predictions**
1. Evaluate predictions and focus on poorly predicted "groups":
  1. Strongest misclassifications. E.g. the model is very confident about the wrong label
  1. Evaluate predictions near decision boundaries.

**EDA and Domain Expertise**
1. Do a thorough EDA: look for feature augmentations that result in linear decision boundaries between pairs of classes.
1. Learn about the domain. Read [Analysis of Identification Method for Bacterial Species and Antibiotic Resistance Genes Using Optical Data From DNA Oligomers](https://www.frontiersin.org/articles/10.3389/fmicb.2020.00257/full) and cited/citing references.